In [7]:
rdd = sc.textFile("./listaAmigos.txt")

linesPairRDD = rdd.map(lambda x: x.split()) \
                .map(lambda x: (int(x[0]), list(map(int, x[1].split(","))) if len(x) == 2 else ()))

# Generate already friends RDD, possible suggestions RDD, and substract second from first
from itertools import combinations

def generateFriendsRDD(line):
    person = line[0]
    friends = line[1]
    
    return [(tuple(sorted((person, friend))), 0) for friend in friends]

def generateCommonFriendsRDD(line):
    friends = line[1]
    
    return [(tuple(sorted(friendsPair)), 1)
            for friendsPair in combinations( friends, 2 )]

friendsRDD = linesPairRDD.flatMap(generateFriendsRDD)
commonFriendsRDD = linesPairRDD.flatMap(generateCommonFriendsRDD)

noFriendsRDD = commonFriendsRDD.subtractByKey(friendsRDD) \
                        .reduceByKey(lambda a,b: a + b)

#Order them using a Max-heap, then take top 10 for each key
import heapq
from pyspark.rdd import RDD
 
def takeOrderedByKey(self, num, sortValue = None):
 
        def init(a):
            return [a]
 
        def combine(agg, a):
            agg.append(a)
            return getTopN(agg)
 
        def merge(a, b):
            agg = a + b
            return getTopN(agg)
 
        def getTopN(agg):
            return heapq.nlargest(num, agg, sortValue)          
 
        return self.combineByKey(init, combine, merge)
        
RDD.takeOrderedByKey = takeOrderedByKey

suggestionsRDD = noFriendsRDD.flatMap(lambda x: ((x[0][0], (x[0][1], x[1])), (x[0][1], (x[0][0], x[1])))) \
                            .takeOrderedByKey(10, sortValue=lambda x: (x[1], -x[0])) \
                            .map(lambda x: (x[0], [x[0] for x in x[1]]))

suggestionsRDD.sortByKey().repartition(1).saveAsTextFile("FriendsSuggestions")